In [18]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127921")
exp = Experiment(workspace=ws, name="quick-starts-ws-127921")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127921
Azure region: southcentralus
Subscription id: a0f586ec-8016-4ea9-8248-9bf2299ad437
Resource group: aml-quickstarts-127921


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"

try:

    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected")

except ComputeTargetException:
    print("New compute target creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_node=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True, max_nodes=4)         
    print("Cluster is ready")                                                             

Existing cluster detected


In [48]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
parameter_sampling = RandomParameterSampling( {
                        "--C":uniform(0.5,1),
                        "--max_iter":choice(20,40,60,80,100) 
}) 

# Specify a Policy
early_termination_policy = BanditPolicy (evaluation_interval=1,slack_factor=0.2)
 
# Create a SKLearn estimator for use with train.py
#import packages
from azureml.core import ScriptRunConfig
from azureml.core import Environment
script='train.py'
est=SKLearn(source_directory='.',entry_script='train.py',compute_target=new_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(estimator=est,
                              hyperparameter_sampling=parameter_sampling,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              policy=early_termination_policy,
                              max_total_runs=4,
                              max_concurrent_runs=4
                                     )

In [49]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run=exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_762ae35c-8fcd-4e1d-945b-149d552eee7a
Web View: https://ml.azure.com/experiments/quick-starts-ws-127921/runs/HD_762ae35c-8fcd-4e1d-945b-149d552eee7a?wsid=/subscriptions/a0f586ec-8016-4ea9-8248-9bf2299ad437/resourcegroups/aml-quickstarts-127921/workspaces/quick-starts-ws-127921

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-25T22:21:34.137711][API][INFO]Experiment created<END>\n""<START>[2020-11-25T22:21:34.735425][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-25T22:21:34.911500][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-25T22:21:37.1176302Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_762ae35c-8fcd-4e1d-945b-149d552eee7a
Web View: https://ml.azure.com/experiments/quick-starts-ws-127921/runs/HD_762ae35c-8fcd-4e1d-945b-149d552eee7a?wsid=/subsc

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=src)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
#splitting the dataset
from sklearn.model_selection import train_test_split
import pandas as pd

x_train, x_test, y_train, y_test=train_test_split(x, y, train_size=0.8, test_size=0.2, random_size=42)
train_data=pd.concat((x_train,y_train),axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=2,
    compute_target=cpu_cluster,
    feauturization:'auto',
    max_concurrent_iterations=4,
    max_cores_per_iteration=4 )

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(AutoML_Config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###